In [1]:
import pandas as pd
import requests
import re
from io import StringIO

In [146]:
signature_matrix_url = 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4739640/bin/NIHMS670442-supplement-2.xlsx'
filename = 'signature_matrix'
try:
    open(f'./data/raw/{filename}.xlsx', 'r')
except FileNotFoundError:
    r = requests.get(signature_matrix_url)
    open(f'./data/raw/{filename}.xlsx', 'wb').write(r.content)
df = pd.read_excel(f'./data/raw/{filename}.xlsx', sheet_name='SuppTable1_GEP_Matrix', header=13, index_col='Gene symbol')
df.drop(['Entrez ID', 'Max probeseta'], axis=1).to_csv(f'./data/{filename}.csv')

/Users/luke/projects/cell-type-deconvolution/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [144]:
FILENAME = 'ground_truth_proportions' 
try:
    open(f'./data/raw/{FILENAME}.xlsx', 'r')
except FileNotFoundError:
    r = requests.get('https://static-content.springer.com/esm/art%3A10.1038%2Fnmeth.3337/MediaObjects/41592_2015_BFnmeth3337_MOESM205_ESM.xlsx')
    open(f'./data/raw/{FILENAME}.xlsx', 'wb').write(r.content)
pd.read_excel(f'./data/raw/{FILENAME}.xlsx').to_csv(f'./data/{FILENAME}.csv', index=False)

signature_matrix = pd.read_csv(f"./data/signature_matrix.csv", index_col=0)
ground_truth = pd.read_csv('./data/ground_truth_proportions.csv').pivot(index='Sample ID', columns='Cell type', values='Flow (%)')
signature_matrix.columns
COL_MAPPING_GT_TO_SIG = {
    'Activated memory CD4 T cells': 'T cells CD4 memory activated', 
    'CD8 T cells': 'T cells CD8',
    'Memory B cells': 'B cells memory',
    'Monocytes': 'Monocytes',
    # TODO: which nk cells? 'NK cells activated', or 'NK cells resting'?
    'NK cells':'NK cells resting', 
    'Naïve B cells': 'B cells naive', 
    'Naïve CD4 T cells':  'T cells CD4 naive',
    'Resting memory CD4 T cells': 'T cells CD4 memory resting',

}
new_ground_truth_cols = [COL_MAPPING_GT_TO_SIG[c] for c in ground_truth.columns]
ground_truth.columns = new_ground_truth_cols
assert sum(ground_truth.columns.isin(signature_matrix.columns) == False ) == 0
ground_truth.to_csv('./data/ground_truth_proportions.csv')

/Users/luke/projects/cell-type-deconvolution/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [22]:
try:
    file = open('./data/raw/HumanHT-12 V4.0 expression beadchip.txt', 'r').read()
except FileNotFoundError:
    r = requests.get('https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?mode=raw&is_datatable=true&acc=GPL10558&id=50081&db=GeoDb_blob135')
    open('./data/raw/HumanHT-12 V4.0 expression beadchip.txt', 'wb').write(r.content)
    file = r.text
raw_data = file.split("#GB_ACC = \n")[1]
beadchip_probe_definitions = pd.read_csv(StringIO(raw_data), sep='\t', index_col=0)0

In [120]:
cell_mix_ids = {
    "GSM1587800": "17-002",
    "GSM1587801": "17-006",
    "GSM1587802": "17-019",
    "GSM1587803": "17-023",
    "GSM1587804": "17-026",
    "GSM1587805": "17-027",
    "GSM1587806": "17-030",
    "GSM1587807": "17-034",
    "GSM1587808": "17-040",
    "GSM1587809": "17-041",
    "GSM1587810": "17-042",
    "GSM1587811": "17-043",
    "GSM1587812": "17-045",
    "GSM1587813": "17-047",
    "GSM1587814": "17-054",
    "GSM1587815": "17-055",
    "GSM1587816": "17-057",
    "GSM1587817": "17-058",
    "GSM1587818": "17-060",
    "GSM1587819": "17-061",
}

dfs = []
for mix_id in cell_mix_ids.keys():
    try:
        html = open(f'./data/raw/cell_mixes/{mix_id}.html', 'r').read()
    except FileNotFoundError:
        html = requests.get(f'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc={mix_id}').text
        open(f'./data/raw/cell_mixes/{mix_id}.html', 'w').write(html)
    split = html.split('#Detection Pval = \n')[1].split('\n')
    assert len(split) == 47_336
    raw_cols = split[0].split('</strong><strong>\t')
    clean = re.compile('<.*?>')
    cols = '\t'.join([re.sub(clean, '', string) for string in raw_cols])
    df = pd.read_csv(StringIO('\n'.join([cols] + split[2:47_326])), sep='\t', dtype=str, index_col=0)
    mix_name = cell_mix_ids[mix_id]
    df = df.rename(columns={'VALUE': mix_name})
    dfs.append(df[mix_name])
all_mixes = pd.concat(dfs, axis=1)

# Map to gene names from probe names using beadchip probe definitions
all_mixes = all_mixes.astype(float)
all_mixes['Gene symbol'] = all_mixes.index.map(lambda i: beadchip_probe_definitions.loc[i]['Symbol'])
all_mixes.dropna(inplace=True, subset='Gene symbol')

In [121]:
# Remove duplicate rows with duplicate Gene symbols, keeping the row with the highest average expression
# From https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4739640:
# "For non-Affymetrix platforms [this one is Illumina], genes mapping to >1 
# probe were collapsed at the gene-level according to the probe with highest 
# mean expression across all samples."

mean_expression_per_probe = all_mixes.drop('Gene symbol', axis=1, inplace=False).T.mean()
best_duplicate_probe_index = pd.concat([mean_expression_per_probe, all_mixes['Gene symbol']], axis=1).sort_values(0).drop_duplicates(subset='Gene symbol', keep='last')
deduped_mixes = all_mixes.loc[best_duplicate_probe_index.index].set_index('Gene symbol').sort_index()

In [135]:
deduped_mixes.to_csv('./data/cell_mixes.csv')